# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [30]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [31]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,2.23,74,82,15.38,TF,1696295114
1,1,puerto natales,-51.7236,-72.4875,2.83,96,100,1.78,CL,1696295115
2,2,anadyr,64.7500,177.4833,4.28,64,0,7.00,RU,1696295115
3,3,as sulayyil,20.4607,45.5779,25.14,15,0,1.67,SA,1696295116
4,4,adamstown,-25.0660,-130.1015,20.41,73,4,9.74,PN,1696295116


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [38]:
%%capture --no-display

# Configure the map plot
city_humidity = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM",frame_width = 800, frame_height = 700,
    size = "Humidity",
    color="City",
    alpha=0.70
).opts(title="Map")


# Display the map
city_humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [33]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 18)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,2.23,74,82,15.38,TF,1696295114
1,1,puerto natales,-51.7236,-72.4875,2.83,96,100,1.78,CL,1696295115
2,2,anadyr,64.7500,177.4833,4.28,64,0,7.00,RU,1696295115
5,5,grytviken,-54.2811,-36.5092,1.52,65,99,4.87,GS,1696295117
6,6,carnarvon,-24.8667,113.6333,17.82,50,0,11.55,AU,1696295118


### Step 3: Create a new DataFrame called `hotel_df`.

In [34]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = pd.DataFrame({"City": city_data_df["City"],
                        "Country": city_data_df["Country"],
                        "Lat": city_data_df["Lat"],
                        "Lng": city_data_df["Lng"],
                        "Humidity": city_data_df["Humidity"]})

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port-aux-francais,TF,-49.3500,70.2167,74,
1,puerto natales,CL,-51.7236,-72.4875,96,
2,anadyr,RU,64.7500,177.4833,64,
5,grytviken,GS,-54.2811,-36.5092,65,
6,carnarvon,AU,-24.8667,113.6333,50,
...,...,...,...,...,...,...
525,yorkton,CA,51.2167,-102.4677,100,
529,trondheim,NO,63.4305,10.3951,94,
535,taltal,CL,-25.4000,-70.4833,78,
536,ishimbay,RU,53.4545,56.0415,63,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [35]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


port-aux-francais - nearest hotel: Keravel
puerto natales - nearest hotel: Dorotea Patagonia Hostel
anadyr - nearest hotel: Гостевой дом
grytviken - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
sima - nearest hotel: у Ок'и
hunsur - nearest hotel: Hotel Devishree Prasad
aasiaat - nearest hotel: SØMA
waitangi - nearest hotel: Hotel Chathams
thompson - nearest hotel: Thompson Inn
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
olonkinbyen - nearest hotel: No hotel found
mirnyy - nearest hotel: Зарница
bilibino - nearest hotel: No hotel found
rio grande - nearest hotel: Hotel Vila Moura Executivo
whitehorse - nearest hotel: Town & Mountain Hotel
coquimbo - nearest hotel: Hotel Iberia
yellowknife - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
longyearbyen - nearest hotel: The Vault
north battleford - nearest hotel: Beaver Hotel
hermanus - nearest hotel: Aloe gue

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port-aux-francais,TF,-49.3500,70.2167,74,Keravel
1,puerto natales,CL,-51.7236,-72.4875,96,Dorotea Patagonia Hostel
2,anadyr,RU,64.7500,177.4833,64,Гостевой дом
5,grytviken,GS,-54.2811,-36.5092,65,No hotel found
6,carnarvon,AU,-24.8667,113.6333,50,No hotel found
...,...,...,...,...,...,...
525,yorkton,CA,51.2167,-102.4677,100,Ramada Yorkton
529,trondheim,NO,63.4305,10.3951,94,Thon Hotel Trondheim
535,taltal,CL,-25.4000,-70.4833,78,Hostería Taltal
536,ishimbay,RU,53.4545,56.0415,63,Заря


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [37]:
%%capture --no-display

# Configure the map plot
hotel_list = hotel_df.hvplot.points("Lng", "Lat", geo = True, tiles = "EsriNatGeo", frame_width = 900, frame_height = 700, size = "Humidity",
    color="City",
    alpha=.70,
    hover_cols=["Hotel Name", "Country"]
).opts(title="Hotels")

# Display the map
hotel_list

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)